In [1]:
# Apply custon style to notebook
from IPython.core.display import HTML
import pathlib
styles_path = pathlib.Path(pathlib.Path().absolute(), "style", "style.css")
styles = open(styles_path, "r").read()
HTML(f"<style>{styles}</style>")

_Note: This notebook must be run locally. Due to how the Swift Simulator operates, this notebook will not run on Google Colab_

# 4.0 Null-Space Projection for Motion Control

$\large{\text{Manipulator Differential Kinematics}} \\ \large{\text{Part II: Acceleration and Advanced Applications}}$

$\text{By Jesse Haviland and Peter Corke}$

<br>

The sections of the Tutorial paper related to this notebook are listed next to each contents entry.
It is beneficial to read these sections of the paper before attempting the notebook Section.

### Contents

[4.1 Swift and Robot Setup](#swift)

[4.2 Null-Space Projection](#nsp)
* Advanced Velocity Control
  * Null-space Projection

[4.3 Manipulability Maximising](#mm)
* Advanced Velocity Control
  * Null-space Projection


In [2]:
# We will do the imports required for this notebook here

# numpy provides import array and linear algebra utilities
import numpy as np

# the robotics toolbox provides robotics specific functionality
import roboticstoolbox as rtb

# spatial math provides objects for representing transformations
import spatialmath as sm

# swift is a lightweight browser-based simulator which comes eith the toolbox
from swift import Swift

# spatialgeometry is a utility package for dealing with geometric objects
import spatialgeometry as sg

<br>

<a id='swift'></a>

### 4.1 Swift and Robot Setup
---

In this notebook we will be using the Robotics Toolbox for Python and Swift to simulate our motion controllers. Check out Part 1 Notebook 3 for an introduction to these packages.

In [3]:
# Make the environment
env = Swift()

# Launch the simulator
# The realtime flag will ask the simulator to simulate as close as
# possible to realtime as apposed to as fast as possible
env.launch(realtime=True, browser="notebook")
env.set_camera_pose([1.3, 0, 0.4], [0, 0, 0.3])

# Make a panda robot
panda = rtb.models.Panda()

# Set the joint coordinates to qr
panda.q = panda.qr

# We can then add our robot to the simulator envionment
env.add(panda)

# end-effector axes
ee_axes = sg.Axes(0.1)

# goal axes
goal_axes = sg.Axes(0.1)

# Add the axes to the environment
env.add(ee_axes)
env.add(goal_axes) 

2

<br>

<a id='nsp'></a>

### 4.2 Null-Space Projection
---

Many modern manipulators are redundant — they have more than 6 degrees of freedom. We can exploit this redundancy by having the robot optimize a performance measure while still achieving the original goal. In this Section, we start with the resolved-rate motion control (RRMC) algorithm explained in Part 1 Notebook 3

\begin{equation*}
     \dot{\bf{q}} = \bf{J}(\bf{q})^{+} \ \bf{\nu}.
\end{equation*}

The Jacobian of a redundant manipulator has a null space. Any joint velocity vector which is a linear combination of the manipulator Jacobian's null-space will result in no end-effector motion ($\bf{\nu} = 0$). We can augment RRMC to add a joint velocity vector $\dot{\bf{q}}_{null}$ which can be projected into the null-space resulting in zero end-effector spatial velocity

\begin{equation*}
    \dot{\bf{q}} =
    \underbrace{
        \bf{J}(\bf{q})^{+} \ \bf{\nu}
    }_{\text{end-effector \ motion}} + \ \
        \underbrace{
        \left(
            \bf{1}_n - \bf{J}(\bf{q})^{+} \bf{J}(\bf{q})
        \right)
        \dot{\bf{q}}_\text{null}
    }_{\text{null-space \ motion}}
\end{equation*}

where $\bf{1}_n \in \mathbb{R}^{n \times n}$ is an identity matrix, and $\dot{\bf{q}}_\text{null}$ is the desired joint velocities for the null-space motion.

We can set $\dot{\bf{q}}_\text{null}$ to be the gradient of any scalar performance measure $\gamma(\bf{q})$ -- the performance measure must be a differentiable function of the joint coordinates $\bf{q}$.

Lets make a Python method to do this

In [4]:
def null_project(robot, q, qnull, ev, λ):
    """
    Calculates the required joint velocities qd to achieve the desired
    end-effector velocity ev while projecting the null-space motion qnull
    into the null-space of the jacobian of the robot.

    robot: a Robot object (must be redundant with robot.n > 6)
    q: the robots current joint coordinates
    qnull: the null-space motion to be projected into the solution
    ev: the desired end-effector velocity (expressed in the base-frame
        of the robot)
    λ: a gain to apply to the null-space motion

    Note: If you would like to express ev in the end-effector frame,
        change the `jacob0` below to `jacobe`
    """

    # Calculate the base-frame manipulator Jacobian
    J0 = robot.jacob0(q)

    # Calculate the pseudoinverse of the base-frame manipulator Jacobian
    J0_pinv = np.linalg.pinv(J0)

    # Calculate the joint velocity vector according to the equation above
    qd = J0_pinv @ ev + (1.0 / λ) * (np.eye(robot.n) - J0_pinv @ J0) @ qnull.reshape(robot.n,)

    return qd

<br>

<a id='mm'></a>

### 4.3 Manipulability Maximising
---

Park [4] proposed using the gradient of the Yoshikawa manipulability index as $\dot{\bf{q}}_\text{null}$. As detailed in Part 1 of this Tutorial, the manipulability index is calculated as

\begin{align*}
    m(\bf{q}) = \sqrt{
        \text{det}
        \Big(
            \tilde{\bf{J}}(\bf{q})
            \tilde{\bf{J}}(\bf{q})^\top
        \Big)
    }
\end{align*}

where $\tilde{\bf{J}}(\bf{q}) \in \mathbb{R}^{3 \times n}$ is either the translational or rotational rows of $\bf{J}(\bf{q})$ causing  $m(\bf{q})$ to describe the corresponding component of manipulability.

Taking the time derivative of $m(\bf{q})$ using the chain rule
\begin{align*}
    \frac{\text{d} \ m(t)}
            {\text{d} t} = 
    \dfrac{1}
            {2m(t)} 
    \frac{\text{d} \ \text{det} \left( \tilde{\bf{J}}(\bf{q}) \tilde{\bf{J}}(\bf{q})^\top \right)}
            {\text{d} t} 
\end{align*}

we can write this as

\begin{align*}
    \dot{m}
    &=
    \bf{J}_m^\top(\bf{q}) \ \dot{\bf{q}}
\end{align*}

where

\begin{equation*}
    \bf{J}_m^\top(\bf{q})
    =
    \begin{pmatrix}
        m \, \text{vec} \left( \tilde{\bf{J}}(\bf{q}) \tilde{\bf{H}}(\bf{q})_1^\top \right)^\top 
        \text{vec} \left( (\tilde{\bf{J}}(\bf{q})\tilde{\bf{J}}(\bf{q})^\top)^{-1} \right) \\
        m \, \text{vec} \left( \tilde{\bf{J}}(\bf{q}) \tilde{\bf{H}}(\bf{q})_2^\top \right)^\top 
        \text{vec} \left( (\tilde{\bf{J}}(\bf{q})\tilde{\bf{J}}(\bf{q})^\top)^{-1} \right) \\
        \vdots \\
        m \, \text{vec} \left( \tilde{\bf{J}}(\bf{q}) \tilde{\bf{H}}(\bf{q})_n^\top \right)^\top 
        \text{vec} \left( (\tilde{\bf{J}}(\bf{q})\tilde{\bf{J}}(\bf{q})^\top)^{-1} \right) \\
    \end{pmatrix}
\end{equation*}

is the manipulability Jacobian $\bf{J}^\top_m \in \R^n$ and where the vector operation $\text{vec}(\cdot) : \R^{a \times b} \rightarrow \R^{ab}$ converts a matrix column-wise into a vector,
and $\tilde{\bf{H}}_i \in \R^{3 \times n}$ is the translational or rotational component (matching the choice of $\tilde{\bf{J}}(\bf{q})$)
of $\bf{H}_i \in \R^{6 \times n}$ which is the $i^{th}$ component of the manipulator Hessian tensor $\bf{H} \in \R^{n \times 6 \times n}$.

The complete equation proposed by Park is

\begin{equation*}
    \dot{\bf{q}} =
    \bf{J}(\bf{q})^{+} \ \bf{\nu} +
    \frac{1}{\lambda} 
    \Big(
        \left(
            \bf{1}_n - \bf{J}(\bf{q})^{+} \bf{J}(\bf{q})
        \right)
        \bf{J_m}(\bf{q})
    \Big)
\end{equation*}

where $\lambda$ is a gain which scales the magnitude of the null-space velocities. This equation will choose joint velocities $\dot{\bf{q}}$ which will achieve the end-effector spatial velocity $\bf{\nu}$ while also improving the translational and/or rotational manipulability of the robot.

Lets make this in Python



In [5]:
def jacobm(robot, q, axes):
    """
    Calculates the manipulability Jacobian. This measure relates the rate
    of change of the manipulability to the joint velocities of the robot.

    q: The joint angles/configuration of the robot
    axes: A boolean list which correspond with the Cartesian axes to
        find the manipulability Jacobian of (6 boolean values in a list)

    returns the manipulability Jacobian
    """

    # Calculate the base-frame manipulator Jacobian
    J0 = robot.jacob0(q)

    # only keep the selected axes of J
    J0 = J0[axes, :]

    # Calculate the base-frame manipulator Hessian
    H0 = robot.hessian0(q)

    # only keep the selected axes of H
    H0 = H0[:, axes, :]

    # Calculate the manipulability of the robot
    manipulability = np.sqrt(np.linalg.det(J0 @ J0.T))

    # Calculate component of the Jacobian
    b = np.linalg.inv(J0 @ J0.T)

    # Allocate manipulability Jacobian
    Jm = np.zeros((robot.n, 1))

    # Calculate manipulability Jacobian
    for i in range(robot.n):
        c = J0 @ H0[i, :, :].T
        Jm[i, 0] = manipulability * (c.flatten("F")).T @ b.flatten("F")

    return Jm

As with RRMC, Park's method will provide the joint velocities for a desired end-effector velocity. As we did in Part 1, we can employ this in a position-based servoing (PBS) controller to get the end-effector to travel towards some desired pose. The PBS scheme is

\begin{align*}
    \bf{\nu} = \bf{k} \bf{e}
\end{align*}

where $\bf{\nu}$ is the desired end-effector spatial velocity to be used.

Lets make a try the PBS scheme while maximising the translational manipulability

In [6]:
# Make a new environment and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.set_camera_pose([1.3, 0, 0.4], [0, 0, 0.3])
env.add(panda)
env.add(ee_axes)
env.add(goal_axes) 

# Change the robot configuration to a ready position
panda.q = [0.21, -0.03, 0.35, -1.90, -0.04, 1.96, 1.36]

# Only the translation axes
trans_axes = [True, True, True, False, False, False]

# Only the rotation aces
rot_axes = [False, False, False, True, True, True]

# All axes
all_axes = [True, True, True, True, True, True]

# Step the sim to view the robot in this configuration
env.step(0)

# A variable to specify when to break the loop
arrived = False

# Specify our timestep
dt = 0.05

Tep = (
    panda.fkine(panda.q)
    * sm.SE3.Tx(-0.1)
    * sm.SE3.Ty(0.6)
    * sm.SE3.Tz(0.4)
)
Tep = Tep.A

# Set the goal axes to Tep
goal_axes.T = Tep

# Set the gain on the manipulability maximisation
λ = 0.1

# Specify the gain for the p_servo method
kt = 1.0
kr = 1.3
k = np.array([kt, kt, kt, kr, kr, kr])

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda.jacob0(panda.q)

    # Calculate the desired null-space motion
    qnull = jacobm(panda, panda.q, trans_axes)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda.fkine(panda.q).A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    # Calculate the required joint velocities and apply to the robot
    panda.qd = null_project(panda, panda.q, qnull, ev, λ)

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)

Lets make a try again while maximising the angular manipulability

In [7]:
# Make a new environment and add objects
env = Swift()
env.launch(realtime=True, browser="notebook")
env.set_camera_pose([1.3, 0, 0.4], [0, 0, 0.3])
env.add(panda)
env.add(ee_axes)
env.add(goal_axes) 

# Change the robot configuration to a ready position
panda.q = [0.21, -0.03, 0.35, -1.90, -0.04, 1.96, 1.36]

# Step the sim to view the robot in this configuration
env.step(0)

# A variable to specify when to break the loop
arrived = False

# Run the simulation until the robot arrives at the goal
while not arrived:

    # Work out the base frame manipulator Jacobian using the current robot configuration
    J = panda.jacob0(panda.q)

    # Calculate the desired null-space motion
    qnull = jacobm(panda, panda.q, rot_axes)

    # The end-effector pose of the panda (using .A to get a numpy array instead of an SE3 object)
    Te = panda.fkine(panda.q).A

    # Calculate the required end-effector velocity and whether the robot has arrived
    ev, arrived = rtb.p_servo(Te, Tep, gain=k, threshold=0.001, method="angle-axis")

    # Calculate the required joint velocities and apply to the robot
    panda.qd = null_project(panda, panda.q, qnull, ev, λ)

    # Update the ee axes
    ee_axes.T = Te

    # Step the simulator by dt seconds
    env.step(dt)